In [1]:
__author__ = 'sharop'

# -*- coding: utf-8 -*-

from __future__ import absolute_import, division, print_function, unicode_literals
import string, numpy as np, getopt, sys, random, time, re, pprint, codecs, json
from pprint import pprint

data =[]
with codecs.open('corpus_entrenamiento.jsonline') as f:
    for line in f:
        data.append(json.loads(line.lower()))
    
data_tt =[]
with codecs.open('token_tag.jsonlines') as tt:
    for line in tt:
        data_tt.append(json.loads(line.lower()))
        
        
token_dict ={}
tag_dict ={}

In [2]:
for item in data_tt:
    if tag_dict.has_key(item['tag']):
        tag_dict[item['tag']]= tag_dict[item['tag']]+1
    else:
        tag_dict[item['tag']]= 1
        
    if token_dict.has_key(item['token']):
        token_dict[item['token']]= token_dict[item['token']]+1
    else:
        token_dict[item['token']]= 1

In [3]:
tags=sorted(tag_dict.keys())
tokens = sorted(token_dict.keys())

#Datos de suavizado
k=1
Ntags = len(tags)
NTokens = len(tokens)

In [4]:
#Construccion de la matriz B
B=[[0L for j in range(len(tags)) ] for i in range(len(tokens)) ]
for doc in data_tt:
    i= tokens.index(doc['token'])    
    j= tags.index(doc['tag'])    
    #if doc['tag']==u'ao':
    #print ((i,j))
    B[i][j] = B[i][j] + 1.0
B = np.array(B)
B = [i/sum(i) for i in B] 

In [5]:
# Calculo Pi 
Pi =[0L for i in range(len(tags))]
for d in data:
    pTag = d['document'][0]['tag']
    idxTag = tags.index(pTag)
    Pi[idxTag] += 1
    #print (pTag,idxTag,Pi[idxTag])
Pi = np.array(Pi)
Pi = (Pi+k)/(sum(Pi)+(k*Ntags))

In [6]:
print (Pi,':',sum(Pi))

[ 0.00546448  0.00546448  0.03551913  0.0136612   0.1284153   0.01912568
  0.03278689  0.01912568  0.00273224  0.00546448  0.00273224  0.00273224
  0.00273224  0.00273224  0.00819672  0.00273224  0.00546448  0.00273224
  0.00273224  0.00273224  0.00273224  0.00273224  0.03278689  0.26775956
  0.02185792  0.01092896  0.00819672  0.0136612   0.00273224  0.00273224
  0.00273224  0.04371585  0.01092896  0.1557377   0.00819672  0.07923497
  0.0273224   0.00273224] : 1.0


In [7]:
A=[[0L for j in range(len(tags)) ] for i in range(len(tags)) ]
## Construccion de matrix A
#Iteramos por cada documento
for document in data:
    # Cada documento contiene un elemento que corresponde al contenido.
    tagp=document['document'][0]['tag']
    for i in xrange (1,len(document['document'])):
        tagi = document['document'][i]['tag']
        #print(tagp,tagi)
        A[tags.index(tagp)][tags.index(tagi)] +=1
        tagp = tagi

A = np.array(A)
A = [(i+k)/(sum(i)+(k*Ntags)) for i in A] 

In [8]:
A=np.matrix(A).transpose()
B=np.matrix(B).transpose()
Pi=np.matrix(Pi).transpose()

In [9]:
obs = ['figura','central','budismo']
states=tags
pi = Pi
tp = A
ep = B

In [10]:
V = [ [0L for st in states]  for ob in obs]
path = [0 for y in states]
    
for y,value in enumerate(states):    
    V[0][y] = pi[y,0] * ep[y,tokens.index(obs[0])]
    path[y] = [y]
        
for t in range(1, len(obs)):
    #V.append()
    newpath=[0 for y in states]
    for y,state in enumerate(states):
        (prop, estado) = max([(V[t-1][y0]*tp[y0,y]*ep[y,tokens.index(obs[t])],y0) for y0, values in enumerate(states)])
        V[t][y] = prop
        newpath[y] = path[estado] + [y]
        
    path = newpath
    
n = len(obs) - 1
(prob, estado) = max((V[n][y], y) for y,val in enumerate(states))
    
[tags[i] for i in path[estado]],path[estado],estado
#return V, path

([u'nc', u'aq', u'nc'], [22, 1, 22], 22)

In [11]:
data


[{u'document': [{u'tag': u'np', u'token': u'c.'},
   {u'tag': u'sp', u'token': u'en'},
   {u'tag': u'da', u'token': u'la'},
   {u'tag': u'nc', u'token': u'ciudad'},
   {u'tag': u'sp', u'token': u'de'},
   {u'tag': u'np', u'token': u'sarnath'},
   {u'tag': u'fc', u'token': u','},
   {u'tag': u'nc', u'token': u'distrito'},
   {u'tag': u'sp', u'token': u'de'},
   {u'tag': u'np', u'token': u'benar\xe9s'},
   {u'tag': u'fc', u'token': u','},
   {u'tag': u'nc', u'token': u'estado'},
   {u'tag': u'sp', u'token': u'de'},
   {u'tag': u'np', u'token': u'uttar_pradesh'},
   {u'tag': u'fc', u'token': u','},
   {u'tag': u'np', u'token': u'india'},
   {u'tag': u'fp', u'token': u'.'}]},
 {u'document': [{u'tag': u'di', u'token': u'otros'},
   {u'tag': u'nc', u'token': u'nombres'},
   {u'tag': u'np', u'token': u'sidarta_gautama'},
   {u'tag': u'fc', u'token': u','},
   {u'tag': u'np', u'token': u'sakiamuni'},
   {u'tag': u'sp', u'token': u'a'},
   {u'tag': u'np', u'token': u'fecha'},
   {u'tag': u'sp',